<a href="https://colab.research.google.com/github/rishvikk11/Acute-Leukemia-Biomarker-Analysis/blob/main/researchcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Step 1: Load your AML merged data and split patient data into three subsets
merged_data = pd.read_csv("//Users/rishvikkambhampati/Desktop/AML Clinical and Gene Expression Data.csv")
gene_data = pd.read_csv("//Users/rishvikkambhampati/Desktop/AML Gene Expression.csv")
death_classification = merged_data["Death Classification"]
patient_classification = merged_data["Sample ID"]
# removed_columns = merged_data.drop['Age at Diagnosis',	'Age At Specimen Acquisition',	'FLT3-ITD Allelic Ratio	ASXL1',	'Cancer Type',	'Cancer Type Detailed',	'Cause Of Death',	'CEBPA bi-allelic',	'Center ID',	'Cohort',	'Cumulative Chemo',	'Unique Regiments',	'No. Of Cumulative Regiments',	'Unique Treatment Stages',	'No. Of Unique Treatment Stages',	'Unique Treatment Types',	'No. Of Cumulative Treatments',	'Current Regimen',	'Current Stage',	'Disease Stage At Specimen Collection',	'Diagnosis At Inclusion',	'ELN2017',	'Ethnicity Category',	'FAB BLAST morphology',	'FLT3-ITD',	'De Novo',	'Relapsed',	'Transformed',	'Karyotype',	'Most Recent Treatment Duration',	'Most Recent Treatment Type',	'Mutation Count',	'Diagnosis non AML, MDS, MPN',	'NPM1',	'Oncotree Code',	'Overall Survival (Months)',	'Overall Survival Status',	'Other Cytogenetics',	'Prior Non-Myeloid',	'Prior Malignancy Radiation',	'Type Non-Myeloid',	'Prior MDS',	'Prior MDS > Two Months',	'Prior MDS/MPN',	'Prior MDS/MPN >Two Months',	'Prior MPN',	'Prior MPN > Two Months',	'Race Category',	'Response Duration To Induction',	'Response To Induction',	'RUNX1',	'Number of Samples Per Patient',	'Sex',	'Diagnosis MDS, MPN',	'Specific Diagnosis At Inclusion',	'Specimen Groups',	'Specimen Type',	'Surface Antigens Immunohistochemical Stains',	'Time Of Sample Collection, Relative To Inclusion',	'TMB (nonsynonymous)',	'TP53 Pathway',	'Treatment Type Induction',	'Used In Manuscript	Variant Summary',	'SAMPLE_ID']
died_before_two_years = []
died_after_two_years = []
alive = []
for i in range(death_classification.size):
    if(death_classification[i] == "Died Before Two Years"):
        died_before_two_years.append(patient_classification[i])
    elif(death_classification[i] == "Died After Two Years"):
        died_after_two_years.append(patient_classification[i])
    else:
        alive.append(patient_classification[i])

updated_data = gene_data.drop("Death Classification", axis = 'columns')
updated_data.pop("SAMPLE_ID")
died_before_two_years_genes = updated_data[merged_data["Death Classification"] == "Died Before Two Years"]
died_after_two_years_genes = updated_data[merged_data["Death Classification"] == "Died After Two Years"]
alive_genes = updated_data[merged_data["Death Classification"] == "Alive"]


X_train, X_test, y_train, y_test = train_test_split(updated_data, gene_data["Death Classification"], test_size=0.4, random_state=42)

model = SVC(kernel = 'linear')
model.fit(X_train, y_train)

coefficients = model.coef_[0]

gene_coefficients = pd.DataFrame(coefficients, columns=["Coefficient"])
list_of_columns = list(updated_data.columns)
gene_coefficients = gene_coefficients.assign(Genes = list_of_columns)


gene_coefficients = gene_coefficients.reindex(gene_coefficients['Coefficient'].abs().sort_values(ascending=False).index)
# model.score(X_train, y_train)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

for col in updated_data:
    if(updated_data[col].dtype == "int64"):
        updated_data[col] = updated_data[col].astype("float64")




import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import recall_score, f1_score, precision_score, accuracy_score
import io



aml_data = pd.read_csv("//Users/rishvikkambhampati/Desktop/AML Gene Expression v4.csv")
all_data = pd.read_csv("//Users/rishvikkambhampati/Desktop/ALL Gene Expression v3.csv")
all_data.pop("SAMPLE_ID")
all_data.pop("Death Classification")

for col in aml_data:
    if(aml_data[col].dtype == "int64"):
        aml_data[col] = aml_data[col].astype("float64")

for col in all_data:
    if(all_data[col].dtype == "int64"):
        all_data[col] = all_data[col].astype("float64")

aml_columns = list(aml_data.columns)
all_columns = list(all_data.columns)
shared_genes = []

for gene in all_columns:
    for gene2 in aml_columns:
        if(gene2 == gene):
            shared_genes.append(gene2)

f_statistics = []
p_values = []
for gene in shared_genes:
    aml = aml_data[gene]
    all = all_data[gene]
    f_statistic, p_value = stats.f_oneway(aml, all)
    f_statistics.append(f_statistic)
    p_values.append(p_value)

anova_frame = pd.DataFrame({"Genes": shared_genes, "F-statistic": f_statistics, "p-value": p_values})
updated_anova_frame = anova_frame.sort_values(by=["F-statistic"], ascending=False)

frames = [aml_data, all_data]
combined_data = pd.concat(frames)

shared_genes.remove("Diagnosis")
X = shared_genes
y = 'Diagnosis'

multi_train_df, multi_test_df = train_test_split(combined_data, test_size = 0.2, random_state = 1)

X_train = multi_train_df[X]
y_train = multi_train_df[y]
X_test = multi_test_df[X]
y_test = multi_test_df[y]

logr = linear_model.LogisticRegression()
logr.fit(X_train,y_train)
pred = logr.predict(X_test)

log_odds = logr.coef_
odds = np.exp(log_odds)


x = combined_data['IDH1']
y = combined_data["Diagnosis"]
sns.regplot(x=x, y=y, data = combined_data, logistic = True, ci = None)

print(odds)
print(accuracy_score(y_test, pred))
print(precision_score(y_test, pred))
print(recall_score(y_test, pred))
print(f1_score(y_test, pred))
print(shared_genes)

